In [1]:
import pandas as pd
import numpy as np
import requests

from pandas.io import gbq
from google.cloud import bigquery,storage
from google.oauth2 import service_account

import gspread

from canvasapi import Canvas

import plotly.express as px

from instances import env_keys
#API_KEY = env_keys['API_key']
API_KEY = env_keys['ACCES_TOKEN']
API_URL = env_keys['API_URL']#+'/accounts/1'

In [2]:
project_id = 'canvas-portal-data-custom'
cred_file = 'canvas-portal-data-custom-6e244db3b826.json'
data_dl = 'data'
scopes = [ "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file",
            "https://spreadsheets.google.com/auth/spreadsheets"]
credentials = service_account.Credentials.from_service_account_file(cred_file,)

In [3]:
# Initialize a new Canvas object
canvas = Canvas(API_URL, API_KEY)
canvas.__dict__

{'_Canvas__requester': <canvasapi.requester.Requester at 0x106d88c88>}

### A super quick start to getting information from Canvas API

For more, see:
https://canvasapi.readthedocs.io/_/downloads/en/latest/pdf/

In [ ]:
#Below are some working examples of how to get the user, course etc. from Canvas
#For more, see https://canvasapi.readthedocs.io/_/downloads/en/latest/pdf/
'''
#1. Get the user object
user = canvas.get_user('xys@mit.edu', 'sis_login_id')

    
#2. Get a course by its id or sis_course_id
course = canvas.get_course(1000) #or, course = canvas.get_course('2019SP15.499', 'sis_course_id')
assn = c_.get_assignments()  #get assignments, or get_files() for files
for a_ in assn[:2]:
    print(a_)


#3. Get the account object, then get the courses in this account
accs = canvas.get_accounts()
for acc_course in accs[0].get_courses()[:10]:
    print(acc_course.name)

type_list_ = ['teacher', 'ta', 'designer']
users = course.get_users(enrollment_type=type_list_)
for user in users:
    print(user)

'''

In [5]:
#Get the LT list from the Excel sheet
lt_df_cols = ['School', 'Dept_num', 'Dept_name', 'Name', 'Email']
lt_df = pd.read_excel('Learning_Technologists_updating.xlsx')
lt_df.columns = lt_df_cols
lt_df['Email'] = lt_df.Email.str.lower()
lt_df.tail()

def get_course_list_by_user(user_email):
    try:
        user_ = canvas.get_user(user_email, 'sis_login_id')
        #print(user_.email, user_.name, user_.id)
        course_list = []
        user_courses = user_.get_enrollments(type=['TeacherEnrollment'])
        for uc_ in user_courses:

            #print(uc_, uc_.id, uc_.course_id,'\n\n')

            c_ = canvas.get_course(uc_.course_id)
            assn_ = c_.get_assignments()
            assn_list = []
            for a_ in assn_:
                assn_list.append(a_)
                
                
            course_list.append([(user_.email).lower(), user_.name, c_.name, c_.id, c_.sis_course_id,
                                c_.course_code, c_.created_at, len(assn_list)])
            
            return course_list
    except Exception as e:
        print('Error {} for user {}'.format(e, user_email))
        return None





course_cols = ['Email', 'name', 'course_name', 'course_id','sis_course_id','course_code',
               'created_at', 'num_assignments']
course_list = []


for u_ in lt_df.Email.tolist():
    returned_list = get_course_list_by_user(u_)
    if returned_list != None:
        course_list.extend(returned_list)

course_df = pd.DataFrame.from_records(course_list, columns=course_cols)

Error Not Found for user melc@mit.edu
Error Not Found for user ajzhou@mit.edu


In [7]:
dept_dict = dict(zip(lt_df.Dept_name, lt_df.Dept_num))
courses_to_exclude = [3157, 3158]
course_df['created_at'] = course_df['created_at'].str[:10]
coruse_created_start = '2020-05-25'
course_df_1 = course_df[(~course_df.course_id.isin(courses_to_exclude)) & 
                        (course_df.created_at>=coruse_created_start)].reset_index(drop=True)
course_df_1['if_sandbox'] = np.where(course_df_1.sis_course_id.str.contains('sand-'), 1, 0)
course_df_1.tail()

,Email,name,course_name,course_id,sis_course_id,course_code,created_at,num_assignments,if_sandbox
36,mmpc2000@mit.edu,Mariela Perez-Cabarcas,Temporary Sandbox,5066,sand-mmpc2000-2021FA,mmpc2000,2020-05-28,6,1
37,kmendez@mit.edu,Keegan Mendez,Countdown to OQE 2020,5050,sand-kmendez-2021FA,kmendez,2020-05-28,8,1
38,glasserk@mit.edu,Kaili Glasser,Temporary Sandbox,4999,sand-glasserk-2021FA,glasserk,2020-05-28,5,1
39,murphyl@mit.edu,Lorcan Murphy,Temporary Sandbox,5074,sand-murphyl-2021FA,murphyl,2020-05-28,6,1
40,ajlwill@mit.edu,Anna Williams,Temporary Sandbox,4983,sand-ajlwill-2021FA,ajlwill,2020-05-28,13,1


In [8]:
lt_course_df = lt_df[['Dept_num','Dept_name','Email']].merge(course_df_1, on=['Email'], how='outer')
lt_course_df_no_sandbox = lt_course_df[lt_course_df.if_sandbox==0]
lt_course_df.to_csv('lt_course_df.csv', index=None)
lt_course_df_no_sandbox.to_csv('lt_course_df_no_sandbox.csv', index=None)
lt_course_df.drop_duplicates(inplace=True)
lt_course_df = lt_course_df.reset_index(drop=True)
lt_course_df.to_gbq('lt_courses.courses_by_lt_all', project_id, if_exists='replace', 
                               credentials=credentials)

lt_grouped = lt_course_df[lt_course_df.if_sandbox==0].groupby(['Email','name','Dept_num', 'Dept_name'], 
                                  as_index=False).agg({'course_id':'count'})
lt_grouped

dept_df_no_sandbox = lt_course_df_no_sandbox.groupby('Dept_name',as_index=False).agg({'course_id':'count'})
dept_df_all = lt_course_df.groupby('Dept_name',as_index=False).agg({'course_id':'count'})
dept_df_no_sandbox.columns = ['Dept_name','num_no_sandbox_courses']
dept_df_all.columns =  ['Dept_name','num_all_courses']
dept_df = dept_df_no_sandbox.merge(dept_df_all, on='Dept_name', 
                        how='outer').fillna(0).sort_values(by='num_no_sandbox_courses', ascending=False)
dept_df['Dept_num'] = dept_df.apply(lambda row: dept_dict[row['Dept_name']], axis=1)
dept_df = dept_df[['Dept_num', 'Dept_name', 'num_no_sandbox_courses','num_all_courses']]
dept_df.to_csv('dept_df.csv', index=None)

#send files to gbq as tables:
dept_df.to_gbq('lt_courses.dept_tally', project_id, if_exists='replace', credentials=credentials)
lt_course_df_no_sandbox.to_gbq('lt_courses.courses_by_lt_no_sandbox', project_id, if_exists='replace', 
                               credentials=credentials)
lt_course_df.to_gbq('lt_courses.courses_by_lt_all', project_id, if_exists='replace', 
                               credentials=credentials)

1it [00:06,  6.54s/it]
1it [00:03,  3.14s/it]
1it [00:03,  3.15s/it]
